# RuleFit Analysis

The goal of the notebook is to use machine learning to automatically create a set of rules to predict a target column.  The rules provide an explainable solution.

## Steps

1. Import and clean data
2. Train rule-fit model
3. Examine rules

## Step 1. Import and Clean Data

In [1]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "12.0.2" 2019-07-16; Java(TM) SE Runtime Environment (build 12.0.2+10); Java HotSpot(TM) 64-Bit Server VM (build 12.0.2+10, mixed mode, sharing)
  Starting server from /Users/megankurka/env/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/fk/z2fjbsq163scfcsq9fhsw7r00000gn/T/tmpyxhp4yeq
  JVM stdout: /var/folders/fk/z2fjbsq163scfcsq9fhsw7r00000gn/T/tmpyxhp4yeq/h2o_megankurka_started_from_python.out
  JVM stderr: /var/folders/fk/z2fjbsq163scfcsq9fhsw7r00000gn/T/tmpyxhp4yeq/h2o_megankurka_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,"14 days, 2 hours and 40 minutes"
H2O cluster name:,H2O_from_python_megankurka_eyjsuh
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"accepting new members, healthy"


In [2]:
df = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv",
                    col_types = {'pclass': "enum", 'survived': "enum"})
df.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,Allen Miss. Elisabeth Walton,female,29,0,0,24160,211.338,B5,S,2,nan,St Louis MO
1,1,Allison Master. Hudson Trevor,male,0.9167,1,2,113781,151.55,C22 C26,S,11,nan,Montreal PQ / Chesterville ON
1,0,Allison Miss. Helen Loraine,female,2,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,0,Allison Mr. Hudson Joshua Creighton,male,30,1,2,113781,151.55,C22 C26,S,nan,135,Montreal PQ / Chesterville ON
1,0,Allison Mrs. Hudson J C (Bessie Waldo Daniels),female,25,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,1,Anderson Mr. Harry,male,48,0,0,19952,26.55,E12,S,3,nan,New York NY
1,1,Andrews Miss. Kornelia Theodosia,female,63,1,0,13502,77.9583,D7,S,10,nan,Hudson NY
1,0,Andrews Mr. Thomas Jr,male,39,0,0,112050,0,A36,S,nan,nan,Belfast NI
1,1,Appleton Mrs. Edward Dale (Charlotte Lamson),female,53,2,0,11769,51.4792,C101,S,nan,nan,Bayside Queens NY
1,0,Artagaveytia Mr. Ramon,male,71,0,0,nan,49.5042,,C,nan,22,Montevideo Uruguay


## Step 2: Train Rule-Fit Model

We will train a rule-fit model to predict the survival.  The outcome of the rulefit model is rules defining whether or not someone will survive.  The rulefit model is done with the following steps:

1. Train a series of random forest models with different depths
2. Extract rules from the random forest models
3. Train a GLM model with Lasso regularization using the rules to predict the target. 
4. Extract the most important rules.

In [54]:
import pandas as pd
import numpy as np
def h2o_rulefit(training_frame, x, y, seed = None, min_depth = 1, max_depth = 10):
    
    family = "gaussian"
    if (training_frame.type(y) == "enum"):
        if training_frame[y].unique().nrow > 2:
            return "Multinomial Not Supported"
        else:
            family = "binomial"
        
    
    # Get paths from random forest models
    paths_frame = training_frame[y]
    depths = range(min_depth, max_depth + 1)
    rf_models = []
    for model_idx in range(len(depths)):
        
        # Train random forest models
        from h2o.estimators.random_forest import H2ORandomForestEstimator
        rf_model = H2ORandomForestEstimator(seed = seed, model_id = "rf.hex", max_depth = depths[model_idx])
        rf_model.train(y = y, x = x, training_frame = training_frame)
        rf_models = rf_models + [rf_model]
    
        paths = rf_model.predict_leaf_node_assignment(training_frame)
        paths.col_names = ["rf_" + str(model_idx) +"."+ x for x in paths.col_names]
        paths_frame = paths_frame.cbind(paths)
    
    # Extract important paths
    from h2o.estimators import H2OGeneralizedLinearEstimator
    glm = H2OGeneralizedLinearEstimator(model_id = "glm.hex", nfolds = 5, seed = seed,
                                        family = family,
                                        alpha = 1, remove_collinear_columns=True,
                                        lambda_search = True)
    glm.train(y = y, training_frame=paths_frame)
    
    import pandas as pd
    rule_importance = pd.DataFrame.from_dict(get_rule_importance(glm), orient = "index").reset_index()
    rule_importance.columns = ["variable", "coefficient"]
    
    # Convert paths to rules
    from h2o.tree import H2OTree
    rules = []
    for i in rule_importance.variable:
        if family == "binomial":
            model_num, tree_num, path = i.replace("rf_", "").replace("T", "").replace("C1.", "").split(".")
        else:
            model_num, tree_num, path = i.replace("rf_", "").replace("T", "").split(".")
        tree = H2OTree(rf_models[int(model_num)], int(tree_num)-1)
        rules = rules + [tree_traverser(tree.root_node, path)]

    # Add rules and order by absolute coefficient
    rule_importance["rule"] = rules
    rule_importance["abs_coefficient"] = rule_importance["coefficient"].abs()
    rule_importance = rule_importance.loc[rule_importance.groupby(["rule"])["abs_coefficient"].idxmax()]  
    rule_importance = rule_importance.sort_values(by = "abs_coefficient", ascending = False)
    rule_importance = rule_importance.drop("abs_coefficient", axis = 1)
    
    return rule_importance


In [55]:
def get_rule_importance(glm):
    from h2o.estimators import H2OGeneralizedLinearEstimator
    
    r = H2OGeneralizedLinearEstimator.getGLMRegularizationPath(glm)
    deviance = r.get('explained_deviance_train')
    inflection_pt = [i*3 for i, x in enumerate(np.diff(np.sign(np.diff(deviance, 2)))) if x != 0 and i > 0][0]
    return {k: v for k,v in r.get('coefficients')[inflection_pt].items() if abs(v) > 0 and k != "Intercept"}

In [56]:
def tree_traverser(node, split_path):
    rule = []
    splits = [char for char in split_path]
    for i in splits:
        if i == "R":
            if np.isnan(node.threshold):
                rule = rule + [{'split_feature': node.split_feature, 
                                'value': node.right_levels, 
                                'operator': 'in'}]
            else:
                rule = rule + [{'split_feature': node.split_feature, 
                                'value': node.threshold, 
                                'operator': '>='}]

            node = node.right_child
        if i == "L":
            if np.isnan(node.threshold):
                rule = rule + [{'split_feature': node.split_feature, 
                                'value': node.left_levels, 
                                'operator': 'in'}]

            else:
                rule = rule + [{'split_feature': node.split_feature, 
                                'value': node.threshold, 
                                'operator': '<'}]

            node = node.left_child
    consolidated_rules = consolidate_rules(rule)
    consolidated_rules = " AND ".join(consolidated_rules.values())
    return consolidated_rules

def consolidate_rules(rules):
    rules = [x for x in rules if x.get("value")]
    features = set([x.get('split_feature') for x in rules])
    consolidated_rules = {}
    for i in features:
        feature_rules = [x for x in rules if x.get('split_feature') == i]
        if feature_rules[0].get('operator') == 'in':
            cleaned_rules = i + " is in " + ", ".join(sum([x.get('value') for x in feature_rules], []))
        else:
            cleaned_rules = []
            operators = set([x.get('operator') for x in feature_rules])
            for op in operators:
                vals = [x.get('value') for x in feature_rules if x.get('operator') == op]
                if '>' in op:
                    constraint = max(vals)
                else:
                    constraint = min(vals)
                cleaned_rules = " and ".join([op + " " + str(round(constraint, 3))])
                cleaned_rules = i + " " + cleaned_rules
        consolidated_rules[i] = cleaned_rules
    
    return consolidated_rules

In [57]:
train, valid = df.split_frame(seed = 1234)

In [58]:
x =  ["age", "sibsp", "parch", "fare", "sex", "pclass"]
rules = h2o_rulefit(train, x, "survived", seed = 1234)

drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%


In [59]:
for i in range(len(rules)):
    print("Coefficient:" + str(round(rules.iloc[i]["coefficient"], 15)) + "\nRule: " + rules.iloc[i]["rule"] + "\n\n")

Coefficient:0.690245852462075
Rule: pclass is in 1, 2 AND sex is in female


Coefficient:-0.558256963724828
Rule: age >= 8.536 AND sex is in male


Coefficient:0.322959683244595
Rule: sex is in female AND sibsp < 2.5


Coefficient:-0.210309346937613
Rule: pclass is in 2, 3 AND sex is in male AND age >= 9.569


Coefficient:-0.13832082963874
Rule: fare < 52.033 AND pclass is in 2, 3 AND sex is in male AND age >= 5.141




There are only 5 rules that are created which are recapped below: 

**Highest Likelihood of Survival**

1. Women in class 1 or 2
2. Women with 2 siblings + spouses or less

**Lowest Likelihood of Survival**
1. Men age 9+
2. Men age 10+ in class 2 or 3
3. Men 6+ in class 2 or 3 with fare < $52

Note: The rules are additive.  That means that if a passenger is described by multiple rules, their probability is added together from those rules.

In [60]:
h2o.cluster().shutdown()

H2O session _sid_abec closed.
